# Bayesian Decision Theory
**Drew Gjerstad**  

_Bayesian Optimization Series_  
[github.com/drewgjerstad/bayesian-optimization](https://github.com/drewgjerstad/bayesian-optimization)

The optimization process in its most basic form is a series of decisions.
Ideally, these decisions are made via a principled approach (i.e.,
strategically) which is where **decision theory** comes into play. Specifically,
the approach used to make such decisions should take into account any available
data when deciding where each observation is made. Unfortunately, it is not
clear how to make these decisions primarily due to the likely incomplete and
ever-changing information about the objective function. 

Previously, we discussed how to use Bayesian inference as a framework that
systematically and quantitatively reasons about the uncertainty in the objective
function. This is one of the main difficulties when making decisions during
optimization since our knowledge of the objective function is only updated from
the outcomes of our own decisions.

In this notebook, we focus on Bayesian decision theory which in effect, "bridges
the gap" between Bayesian inference and decision making in optimization. This
principled approach allows us to make decisions using a probabilistic belief
about the objective function to guide optimization policies under uncertainty.

At the heart of the optimization process is the _optimization policy_ which
determines where we will make an observation next (for the time being, ignoring
the question of termination), acquires the next observation, and updates our
knowledge of the objective. Therefore, we aim to obtain the **optimal policy**
with optimal referring to maximizing the expected utility and quality of the
observed data.

While the idea of deriving this optimal policy may seem simple, especially when
deriving it in a theoretical manner, the theoretically optimal policy is often
impossible to compute and has little practical value. Regardless, the process of
deriving this policy will enable us to see how we can obtain effective
_approximations_.

Coming back to the question of termination, this question itself represents a
decision that is crucial in several applications. A **stopping rule** is a
procedure that decides whether to terminate or continue optimization based on
the observed data. In many cases, this rule is _deterministic_ meaning it is
fixed and known before we begin optimizing. One example is a preallocated search
budget defined by the maximum number of allowed observations. This type of
stopping rule will terminate the optimization once we obtain the maximum number
of allowed observations, regardless of our progress.

Alternatively, we may want to consider the optimization progress (i.e., our
understanding of the objective function and the expected cost of continuing)
when deciding whether to terminate or not. This is a more _dynamic_ stopping
rule that will require more subtle, adaptive stopping rules. We will discuss
this more later and how its formulation inspires better approximations.

## Defining Optimization Policies

To define an optimization policy, we typically use an intermediate function
called the **acquisition function** that will score each observation candidate
based on its utility to aiding the optimization process. Then, the policy can
be defined to observe the point deemed to be most useful (or most "promising")
by the acquisition function. Such a definition is used by nearly all Bayesian
optimization policies, with some literature (as noted by Garnett) using the term
"acquisition function" interchangeably with "policy".

When using the Bayesian approach, the acquisition function is almost always
defined by obtaining the posterior belief (distribution) of the objective given
the data and then defining our preferences for the next observation with respect
to this belief. Using the notation from Garnett's book, we will denote
$\alpha(x;\mathcal{D})$ for a general acquisition function with the data,
$\mathcal{D}$, serving as parameters that shape our preferences.

In more mathematical terms, an acquisition function $\alpha$ defines preferences
over candidate observations by "inducing a total order over the domain". This
means that given existing data $\mathcal{D}$, observing candidate $x$ is
preferred over another candidate $x^\prime$ if
$\alpha(x;\mathcal{D}) > \alpha(x^\prime;\mathcal{D})$.  Rationally, the action
we will prefer is one that maximizes the acquisition function:

\begin{equation*}
    x \in \argmax_{x^\prime\in\mathcal{X}} \alpha(x^\prime;\mathcal{D})
\end{equation*}

We can use the formulation above as a kind of "sub-optimization problem". Once
it is solved, the acquisition function will map a set of observed data to a
candidate $x \in \mathcal{X}$ to observe next, filling the exact role of an
optimization policy.

If you are thinking that the idea of solving global optimization problems by
repeatedly solving global optimization problems is unintuitive, don't worry! In
many cases, this paradox is resolved by the fact that common acquisition
functions have properties making their optimization much more tractable than the
primary optimization problem we are aiming to solve.

Commonly used acquisition functions are both inexpensive to evaluate and are
analytically differentiable which means we can use pre-defined optimizers while
computing the policy formulated above. However, recall that our objective
function is assumed to be rather expensive to evaluate and lacks efficient (if
any at all) gradients. Using the ideas outlined here, we are able to moderate
a difficult problem to several simpler problems, a reasonable first step!

## Formalizing Bayesian Decision Theory

Bayesian decision theory is a framework that we can use to make decisions under
uncertainty while still being flexible enough that it can be applied to nearly
any problem. Here, we introduce Bayesian decision theory in the same manner as
Garnett: focusing on the key concepts through the lense of optimization rather
than unloading the entire theory abstractly. Garnett recommends the following
supplementary texts for a more thorough and in-depth review of the theory:
 * _Optimal Statistical Decisions_ by M. H. DeGroot
 * _Statistical Decision Theory and Bayesian Analysis_ by J. O. Berger

Being sufficiently familiar with this topic can help you understand key concepts
in Bayesian optimization that are examined in the literature less thoroughly
than they perhaps should be. In particular, this topic, as Garnett puts it,
serves as the "hidden origin" of several typical acquisition functions.

Following from Garnett's text, we start with using the Bayesian decision theory
approach for decision making and examine the case of making a single, isolated
decision to see how the framework is used to make optimal decisions. Then, we
will extend this reasoning to make several, or a sequence of, decisions.

### Case 1: Single, Isolated Decisions

There are two defining characteristics of a decision problem under uncertainty:
the action space and the presence of uncertain elements in the environment. We
will review these characteristics first.

The **action space** $\mathcal{A}$ is the set of all available decisions. Keep
in mind that the task at hand is to select an action from this space. In the
context of sequential optimization, we are selecting a point in the domain
$\mathcal{X}$ to observe so we have that $\mathcal{A}=\mathcal{X}$.

The **presence of uncertain elements** in the environment will inherently
influence the results of our actions which complicates our decision. Using
Garnett's notation, let $\psi$ denote a random variable that encompasses any
relevant uncertain elements when making and evaluating a decision. While we may
not have all the information about the uncertainty, we can use Bayesian
inference to reason about $\psi$ given the observed data using the posterior
distribution $p(\psi\vert\mathcal{D})$. We can use this belief to aid our
decision.

Suppose that now we need to make a decision (selected from the action space,
$\mathcal{A}$) under the uncertainty in $\psi$, and informed by observed data
$\mathcal{D}$. We need some way to guide our decision selection process: a
_utility function_.

A real-valued **utility-function** $u(a, \psi, \mathcal{D})$ is used to guide
our choice by measuring the quality of choosing action $a$ if the true state of
the environment is $\psi$, with higher utilities being preferred since the
higher the utility score, the more favorable the outcome. Notice that the
arguments provided to the utility function are all that is required to judge the
quality of a decision:
 * the proposed action $a$
 * observed data informing our current knowledge $\mathcal{D}$
 * uncertain elements missing from our knowledge $\psi$

Since we have incomplete information about $\psi$, we are unable to know the
exact utility of selecting any given action. However, we can compute the
_expected_ utility of selecting an action $a$ based on our posterior belief:

\begin{equation*}
    \mathbb{E}\left[u(a,\psi,\mathcal{D})\vert a,\mathcal{D}\right] =
    \int u(a,\psi,\mathcal{D})p(\psi\vert\mathcal{D})d\psi
\end{equation*}

The expected utility above maps each action to a real value that induces a total
order and provides a simple method to make our decision. We then select an
action that maximizes the expected utility:

\begin{equation*}
    a \in \argmax_{a^\prime\in\mathcal{A}}\mathbb{E}\left[
        u(a^\prime,\psi,\mathcal{D})\vert a^\prime,\mathcal{D}\right]
\end{equation*}

By using this approach, the decision is considered to be optimal as there are no
other actions that would result in greater expected utility. Furthermore, this
method of selecting actions optimally under uncertainty is the central concept
of Bayesian decision making.

### Case 2: Sequential Decisions

## References

bayes opt. by garnett
bayes opt theory and practice by liu